In [9]:
#import libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import sys
import os

In [10]:
#load data
file ="incidencies_comptadors_intelligents.parquet"
data_dir = "../data/"

sample_path=os.path.join(data_dir,f"{file}")


df_ICI=pd.read_parquet(sample_path)
print(f"Correctly loaded ({len(df_ICI):,} rows, {len(df_ICI.columns)} columns)")


Correctly loaded (17,112,709 rows, 12 columns)


In [11]:
# Mean Absolute Error (MAE)
def mae(y_true, y_pred):
    #average difference between predicted and actual values.
    return mean_absolute_error(y_true, y_pred)



In [12]:
#root mean squared error
def rmse(y_true, y_pred):
    #measures the average magnitude of the errors between predicted and actual values
    mse = mean_squared_error(y_true, y_pred)
    return np.sqrt(mse)

In [13]:
# mean absolute percentage error 
def mape(y_true, y_pred):
    #percentage error relative to actual values.
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    
    # avoid division by zero
    mask = y_true != 0
    if not np.any(mask):
        return 0.0
    
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100


In [14]:
#mean bias error
def mean_bias_error(y_true, y_pred):
    #average bias in predictions
    return np.mean(y_pred - y_true)

In [21]:
def generate_evaluation_report(y_true, y_pred, model_name="Model"):
    #dictionary to store metrics and print them as a report 
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    
    metrics = {
        'MAE': mae(y_true, y_pred),
        'RMSE': rmse(y_true, y_pred),
        'MAPE': mape(y_true, y_pred),
        'MBE': mean_bias_error(y_true, y_pred)
    }
    
    print(f"\n{'='*60}")
    print(f"EVALUATION METRICS: {model_name}")
    print(f"{'='*60}")
    print(f"\nRegression Metrics (for {len(y_true)} predictions):\n")
    print(f"  MAE (Mean Absolute Error):     {metrics['MAE']:>10.4f}")
    print(f"  RMSE (Root Mean Squared Error): {metrics['RMSE']:>10.4f}")
    print(f"  MAPE (Mean Absolute % Error):  {metrics['MAPE']:>10.2f}%")
    print(f"  MBE (Mean Bias Error):         {metrics['MBE']:>10.4f}")
    print(f"{'='*60}\n")
    
    return metrics

In [16]:
#FUNCTIONS FROM 1.3.3
def predict_next_month_total_consumption(df_poliza, poliza_id, forecast_days=30):   
    # Feature engineering
    df_poliza["lag_1"] = df_poliza["CONSUMO_REAL"].shift(1)
    df_poliza["lag_7"] = df_poliza["CONSUMO_REAL"].shift(7)
    df_poliza["rolling_mean_7"] = (
        df_poliza["CONSUMO_REAL"].shift(1).rolling(window=7).mean()
    )
    df_poliza = df_poliza.dropna().reset_index(drop=True)
    
    # Model training
    features = ["year", "month", "day", "dayofweek", "lag_1", "lag_7", "rolling_mean_7"]
    target = "CONSUMO_REAL"
    
    X = df_poliza[features]
    y = df_poliza[target]
    
    model = XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        tree_method="hist"
    )
    model.fit(X, y)
    
    # Forecasting
    last_known = df_poliza.iloc[-1].copy()
    forecast = []
    history = df_poliza.copy()
    
    for i in range(1, forecast_days + 1):
        next_date = last_known["FECHA"] + pd.Timedelta(days=i)
        
        # Compute features dynamically
        recent_data = history.tail(7)["CONSUMO_REAL"]
        new_data = {
            "year": next_date.year,
            "month": next_date.month,
            "day": next_date.day,
            "dayofweek": next_date.dayofweek,
            "lag_1": history.iloc[-1]["CONSUMO_REAL"],
            "lag_7": recent_data.iloc[0] if len(recent_data) >= 7 else history.iloc[-1]["CONSUMO_REAL"],
            "rolling_mean_7": recent_data.mean()
        }
        
        X_future = pd.DataFrame([new_data])[features]
        next_consumption = float(model.predict(X_future)[0])
        
        new_row = {
            "POLIZA_SUMINISTRO": poliza_id,
            "FECHA": next_date,
            "CONSUMO_REAL": next_consumption,
            "year": next_date.year,
            "month": next_date.month,
            "day": next_date.day,
            "dayofweek": next_date.dayofweek,
            "is_forecast": True
        }
        
        forecast.append(new_row)
        history = pd.concat([history, pd.DataFrame([new_row])], ignore_index=True)
    
    forecast_df = pd.DataFrame(forecast)
    df_poliza["is_forecast"] = False
    df_extended = pd.concat([df_poliza, forecast_df], ignore_index=True).sort_values("FECHA")
    total_consumption = forecast_df["CONSUMO_REAL"].sum()
    
    return total_consumption, forecast_df, df_extended, model


def call_predict_next_month_total_consumption(df, poliza_id, forecast_days=30):
    
    df_ = df[["POLIZA_SUMINISTRO", "FECHA", "CONSUMO_REAL"]].copy()
    
    # Filter for given POLIZA
    df_poliza = df_[df_["POLIZA_SUMINISTRO"] == poliza_id].copy()
    if df_poliza.empty:
        raise ValueError(f"No data found for POLIZA_SUMINISTRO = {poliza_id}")
    
    df_poliza["FECHA"] = pd.to_datetime(df_poliza["FECHA"])
    df_poliza = df_poliza.sort_values(by="FECHA").reset_index(drop=True)
    
    # Add temporal features
    df_poliza["year"] = df_poliza["FECHA"].dt.year
    df_poliza["month"] = df_poliza["FECHA"].dt.month
    df_poliza["day"] = df_poliza["FECHA"].dt.day
    df_poliza["dayofweek"] = df_poliza["FECHA"].dt.dayofweek
    
    total_consumption, forecast_df, df_extended, model = predict_next_month_total_consumption(
        df_poliza, poliza_id, forecast_days
    )
    
    return total_consumption, forecast_df, df_extended, model

In [17]:
def evaluate_model(df_train, df_test, poliza_id, forecast_days=30):
   #evaluate the model using training and test datasets
    print(f"\n{'='*70}")
    print(f"FORECASTING MODEL EVALUATION")
    print(f"{'='*70}")
    print(f"\nPolicy ID: {poliza_id}")
    print(f"Training period: {df_train['FECHA'].min()} to {df_train['FECHA'].max()}")
    print(f"Test period: {df_test['FECHA'].min()} to {df_test['FECHA'].max()}")
    print(f"Forecast horizon: {forecast_days} days")
    
    # Train model on training data
    print(f"\n[1/3] Training model on {len(df_train)} samples...")
    total_pred_train, forecast_df_train, _, model = call_predict_next_month_total_consumption(
        df_train, poliza_id, forecast_days
    )
    print(f"      ✓ Model trained")
    
    # Get actual vs predicted for test period
    print(f"\n[2/3] Making predictions on test period...")
    y_true = df_test["CONSUMO_REAL"].values
    y_pred = forecast_df_train["CONSUMO_REAL"].values[:len(df_test)]  # align lengths
    
    if len(y_pred) > len(y_true):
        y_pred = y_pred[:len(y_true)]
    elif len(y_pred) < len(y_true):
        y_true = y_true[:len(y_pred)]
    
    print(f"      ✓ Generated {len(y_pred)} predictions")
    
    # Compute metrics
    print(f"\n[3/3] Computing evaluation metrics...")
    metrics = generate_evaluation_report(y_true, y_pred, f"XGBoost Forecaster (Policy {poliza_id})")
    
    # Additional insights
    print(f"\nAdditional Insights:\n")
    print(f"  Actual consumption (test period):")
    print(f"    - Mean: {np.mean(y_true):>10.2f} liters/day")
    print(f"    - Min:  {np.min(y_true):>10.2f} liters/day")
    print(f"    - Max:  {np.max(y_true):>10.2f} liters/day")
    print(f"    - Std:  {np.std(y_true):>10.2f} liters/day")
    print(f"\n  Predicted consumption (test period):")
    print(f"    - Mean: {np.mean(y_pred):>10.2f} liters/day")
    print(f"    - Min:  {np.min(y_pred):>10.2f} liters/day")
    print(f"    - Max:  {np.max(y_pred):>10.2f} liters/day")
    print(f"    - Std:  {np.std(y_pred):>10.2f} liters/day")
    
    # Error analysis
    errors = y_true - y_pred
    print(f"\n  Error Analysis:\n")
    print(f"    - Mean error:      {np.mean(errors):>10.2f} liters/day")
    print(f"    - Median error:    {np.median(errors):>10.2f} liters/day")
    print(f"    - Std of errors:   {np.std(errors):>10.2f} liters/day")
    print(f"    - Max overpredict: {np.max(errors):>10.2f} liters/day")
    print(f"    - Max underpredict: {np.min(errors):>10.2f} liters/day")
    
    return {
        'metrics': metrics,
        'y_true': y_true,
        'y_pred': y_pred,
        'model': model,
        'forecast_df': forecast_df_train
    }



In [23]:
def main():    
    print("\n" + "="*70)
    print("WATER CONSUMPTION FORECASTING - MODEL EVALUATION")
    print("="*70)
    
    print("\n[SETUP] Creating synthetic dataset for demonstration...")
    
    np.random.seed(42)
    dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
    n_samples = len(dates)
    
    # Synthetic water consumption with seasonal pattern
    base_consumption = 100
    seasonal_pattern = 30 * np.sin(np.arange(n_samples) * 2 * np.pi / 365)
    noise = np.random.normal(0, 10, n_samples)
    consumo_real = base_consumption + seasonal_pattern + noise
    consumo_real = np.maximum(consumo_real, 10)  # Ensure positive values
    
    df = pd.DataFrame({
        'POLIZA_SUMINISTRO': ['POL_001'] * n_samples,
        'FECHA': dates,
        'CONSUMO_REAL': consumo_real
    })
    
    print(f"      ✓ Dataset created: {len(df)} daily records")
    print(f"      ✓ Date range: {df['FECHA'].min()} to {df['FECHA'].max()}")
    
    # Split train/test (80/20 split, last 30% as test)
    split_idx = int(len(df) * 0.70)
    df_train = df.iloc[:split_idx].reset_index(drop=True)
    df_test = df.iloc[split_idx:split_idx+30].reset_index(drop=True)  # Test on 30 days
    
    poliza_id = 'POL_001'
    forecast_days = len(df_test)
    
    # Run evaluation
    results = evaluate_model(df_train, df_test, poliza_id, forecast_days=forecast_days)
    
    return results


In [24]:
if __name__ == "__main__":
    results = main()


WATER CONSUMPTION FORECASTING - MODEL EVALUATION

[SETUP] Creating synthetic dataset for demonstration...
      ✓ Dataset created: 365 daily records
      ✓ Date range: 2023-01-01 00:00:00 to 2023-12-31 00:00:00

FORECASTING MODEL EVALUATION

Policy ID: POL_001
Training period: 2023-01-01 00:00:00 to 2023-09-12 00:00:00
Test period: 2023-09-13 00:00:00 to 2023-10-12 00:00:00
Forecast horizon: 30 days

[1/3] Training model on 255 samples...
      ✓ Model trained

[2/3] Making predictions on test period...
      ✓ Generated 30 predictions

[3/3] Computing evaluation metrics...

EVALUATION METRICS: XGBoost Forecaster (Policy POL_001)

Regression Metrics (for 30 predictions):

  MAE (Mean Absolute Error):        12.7084
  RMSE (Root Mean Squared Error):    14.9824
  MAPE (Mean Absolute % Error):       20.07%
  MBE (Mean Bias Error):             5.9131


Additional Insights:

  Actual consumption (test period):
    - Mean:      69.62 liters/day
    - Min:       38.20 liters/day
    - Max: 